## Read .ghg metadata

In [3]:
import preProcessing
import importlib
import time
importlib.reload(preProcessing)
T1 = time.time()

pr = preProcessing.read_ALL('BB',2023,3,processes=2,reset=0)
# pr = preProcessing.read_ALL('BB',2023,3,processes=4,reset=1)
T2 = time.time()
print('\n')
print('Runtime: ',(T2-T1)/60,' minutes')


[]eprocessing Files[████████████████████████████████████████████████████████████] 63/63


Runtime:  0.33419662714004517  minutes


In [55]:
pr.files.loc[pr.files['MetaDataFile'].str.contains('.metadata')].shape

(48, 5)

In [57]:
import setupEP as eP
import importlib
import time
importlib.reload(eP)

########## Note - incomplete ghg files crash program

T1 = time.time()
mR = eP.makeRun('ep_Templates/DefaultSettings.eddypro','BB')
mR.runDates(['2023-03-02 10:00','2023-03-02 18:30'],'test',Processes=2,priority = 'high priority')
T2 = time.time()
print('\n')
print('Runtime: ',(T2-T1)/60)


Running EddyPro[████████████████████████████████████████████████████████████] 2/2

Runtime:  1.4841146032015482


# Speedtests

All times in minutes

### 4 threads (8 timesteps each)

Normal priority: 2.05
High priority: 1.91

### 4 threads (48 timesteps each x 12 days)

High priority: 29

### Eddypro GUI (1 run [12 days, 576 timesteps])

High priority: 90